In [2]:
from rapidfuzz import process as pr
import pandas as pd
import numpy as np

### RapidFuzz

In [19]:
# #load set of DW keywords before fuzzyWuzzy into the file
uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv', names = ['ind', 'keyword'], header=0) 
#unique_keywords=uni_kw

In [20]:
uni_kw

,ind,keyword
0,0,malware
1,1,state repression
2,2,climate consensus
3,3,Canary Islands
4,4,Angela I
...,...,...
32699,32699,nuclear arms
32700,32700,processionary caterpillars
32701,32701,Deutsche Kammerphilharmonie Bremen
32702,32702,Maduro


In [21]:
df = pd.read_csv('../data/interim/clean_keywords_2019-2021_after_FuzzyWuzzy.csv') 


In [45]:
import json

filepath = '../data/interim/clean_keywords_2019-2021_after_RapidFuzz.json'

df = pd.read_json(filepath, orient ='split', compression = 'infer')


In [46]:
df

,lastModifiedDate,keywordStrings,keywordStringsCleanAfterFuzz
0,2019-01-01T03:57:28.904Z,"[NASA, OSIRIS-REx, Bennu, asteroid]","[nasa, osiris-rex, bennu, asteroid]"
1,2019-01-01T06:11:50.527Z,"[English Channel, migration, boats, illegal im...","[english channel, migration, boats, illegal im..."
2,2019-01-01T06:14:35.563Z,"[Brazil, Jair Bolsonaro, Chicago economics, Ha...","[brazil, jair bolsonaro, chicago economics, ha..."
3,2019-01-01T08:26:11.599Z,"[Japan, Tokyo, Harajuku, attack]","[japan, tokyo, harajuku, attack]"
4,2019-01-01T09:05:00.736Z,"[Asia, Bangladesh, elections, Kamal Hossain, S...","[asia, bangladesh, elections, kamal hossain, s..."
...,...,...,...
33825,2021-12-31T19:06:43.423Z,"[Germany, Olaf Scholz, New Year, New Year's ad...","[germany, olaf scholz, new year, new year's ad..."
33826,2021-12-31T20:27:51.092Z,"[Colorado, wildfires, Boulder County, evacuati...","[colorado, wildfires, boulder county, evacuati..."
33827,2021-12-31T20:32:20.303Z,"[RCEP, Regional Comprehensive Economic Partner...","[rcep, regional comprehensive economic partner..."
33828,2021-12-31T20:38:11.201Z,"[France, birth control, contraception, contrac...","[france, birth control, contraception, contrac..."


In [28]:
df.dtypes

lastModifiedDate                object
keywordStrings                  object
keywordStringsCleanAfterFuzz    object
dtype: object

In [44]:
import itertools # to flatten lists of lists

list(itertools.chain(*list(df['keywordStringsCleanAfterFuzz'])))

['nasa',
 'osiris-rex',
 'bennu',
 'asteroid',
 'english channel',
 'migration',
 'boats',
 'illegal immigration',
 'brazil',
 'jair bolsonaro',
 'chicago economics',
 'hamilton mourao',
 'paulo guedes',
 'japan',
 'tokyo',
 'harajuku',
 'attack',
 'asia',
 'bangladesh',
 'elections',
 'kamal hossain',
 'sheikh hasina',
 'awami league',
 'khaleda zia',
 'iran',
 'sanctions',
 'civil society',
 'nuclear deal',
 'jcpoa',
 'us sanctions',
 'tehran',
 'hassan rouhani',
 'ayatollah khomeini',
 'liviu dragnea',
 'viorica dancila',
 'klaus iohannis',
 'romania',
 'bucharest',
 'eu',
 'counterfeit alcohol',
 'adulterated alcohol',
 'fake booze',
 'food fraud',
 'edgar hilsenrath',
 'schriftsteller',
 'jude',
 'der nazi & der friseur',
 'heinrich böll',
 'indonesia',
 'java',
 'landslide',
 'natural disasters',
 'ironman',
 'hawaii',
 'patrick lange',
 "new year's",
 'countdown',
 'solutions',
 'germany',
 'münster',
 'syria',
 'iran',
 'middle east',
 'arab league',
 'gulf arab states',
 'asia

In [14]:
df

,id,name,shortTitle,text,teaser,shortTeaser,kicker,regions,keywords,keywordStrings,...,isOpinion,geographicLocations,contentAssociations,mainContentImageLink,images,externalLinks,topStory,language,ts_lastModifiedDate,keywordStringsClean
0,46912921,"NASA spacecraft enters asteroid Bennu's orbit,...","NASA craft enters asteroid orbit, sets records",<p>NASA's unmanned spacecraft OSIRIS-REx enter...,The OSIRIS-REx spacecraft had arrived at the l...,NASA said its OSIRIS-REx spacecraft succesfull...,NaN,[],"[{'name': 'NASA'}, {'name': 'OSIRIS-REx'}, {'n...","['NASA', 'OSIRIS-REx', 'Bennu', 'asteroid']",...,False,[],"[{'id': 47376129}, {'id': 47376131}, {'id': 47...",{'description': None},[{'name': 'Sonde «Osiris Rex» nach zwei Jahren...,[{'name': 'DW Newsletter'}],False,NaN,2019-01-01 03:57:28.904000+00:00,"['nasa', 'osiris-rex', 'bennu', 'asteroid']"
1,46911356,UK brings in boats to patrol English Channel a...,UK deploys boats amid Channel migration spike,<p>Britain will recall two overseas border pat...,The UK is withdrawing patrol ships from overse...,The UK is withdrawing patrol ships from overse...,NaN,[],"[{'name': 'English Channel'}, {'name': 'migrat...","['English Channel', 'migration', 'boats', 'ill...",...,False,[],"[{'id': 47375206}, {'id': 47375208}, {'id': 47...",{'description': None},[{'name': 'Großbritanien | Zahl der Bootsflüch...,[],False,NaN,2019-01-01 06:11:50.527000+00:00,"['english channel', 'migration', 'boats', 'ill..."
2,46909694,"In Brazil, Jair Bolsonaro tries to unite the e...",Can Bolsonaro unite Brazil's motley right?,<p>Politics without ideologies and party cliqu...,Brazil is inaugurating President Jair Bolsonar...,"In his campaign, Jair Bolsonaro targeted appea...",NaN,[],"[{'name': 'Brazil'}, {'name': 'Jair Bolsonaro'...","['Brazil', 'Jair Bolsonaro', 'Chicago economic...",...,False,[],"[{'id': 47373913}, {'id': 47374152}, {'id': 47...",{'description': None},[{'name': 'Brasilien - Präsident Jair Bolsonar...,[{'name': ''}],False,NaN,2019-01-01 06:14:35.563000+00:00,"['brazil', 'jair bolsonaro', 'chicago economic..."
3,46912694,Tokyo: Man deliberately drives van into NYE cr...,Man drives car into Tokyo NYE crowd,<p>At least nine people were injured when a ma...,"A man with an ""intent to murder"" has driven a ...","A man with an ""intent to murder"" has driven a ...",NaN,[],"[{'name': 'Japan'}, {'name': 'Tokyo'}, {'name'...","['Japan', 'Tokyo', 'Harajuku', 'attack']",...,False,[],"[{'id': 47376009}, {'id': 47376011}, {'id': 47...",{'description': None},[{'name': 'Japan | Autoangriff in Tokyo'}],[],False,NaN,2019-01-01 08:26:11.599000+00:00,"['japan', 'tokyo', 'harajuku', 'attack']"
4,46910092,Bangladesh opposition leader: International co...,International community 'shouldn't recognize n...,"<div class=""vjs-wrapper embed big ""><h2 aria-l...","In an exclusive interview with DW, Kamal Hossa...",Bangladeshi opposition leader Kamal Hossain sl...,NaN,[],"[{'name': 'Asia'}, {'name': 'Bangladesh'}, {'n...","['Asia', 'Bangladesh', 'elections', 'Kamal Hos...",...,False,[],"[{'id': 47374183}, {'id': 47374251}, {'id': 47...",{'description': None},"[{'name': 'Dr. Kamal Hossain '}, {'name': 'Pre...",[],False,NaN,2019-01-01 09:05:00.736000+00:00,"['asia', 'bangladesh', 'elections', 'kamal hos..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33825,60299904,Germany's Olaf Scholz urges unity in first New...,Germany's Scholz urges unity in first New Year...,"<div class=""vjs-wrapper embed big ""><h2 aria-l...","In his first New Year's message, Olaf Scholz h...","""We will be able to master the major changes o...",NaN,[{'name': 'Germany'}],"[{'name': 'Germany'}, {'name': 'Olaf Scholz'},...","['Germany', 'Olaf Scholz', 'New Year', ""New Ye...",...,False,NaN,"[{'id': 59559319}, {'id': 59559321}, {'id': 59...",{'description': 'German Chancellor Olaf Scholz...,[{'name': 'Deutschland Berlin | Neujahrsanspra...,[],True,ENGLISH,2021-12-31 19:06:43.423000+00:00,"['germany', 'olaf schol

In [5]:
print(len(uni_kw))
uni_kw.head()

32704


,ind,keyword
0,0,malware
1,1,state repression
2,2,climate consensus
3,3,Canary Islands
4,4,Angela I


In [134]:
keywords = list(uni_kw['keyword'])
keywords_clean = list(set(uni_kw['keyword'].str.lower())) # Cleaning: Write all in lower case

print(len(keywords))
print(len(keywords_clean))

32704
30385


In [136]:
# Identify all single keywords that contain commas, splits them in mutliple keywords and saves this in a list
new_keywords = []
list_kw_todel = []
substring = ', '
for i,s in enumerate(keywords_clean):
    if substring in s:
        #print('row', i, ':',  s)
        list_kw_todel.append(s)
        new_keywords.append(keywords_clean[i].split(substring))

flat_new_keywords = [item for sublist in new_keywords for item in sublist]

# Remove the non-seperated keywords
for el in list_kw_todel:
    keywords_clean.remove(el)

# Add the seperated ones
keywords_clean = keywords_clean + flat_new_keywords

print(len(keywords_clean))

30405


In [116]:
keywords_clean

['sankt augustin',
 'pratt & whitney',
 'daca',
 'iraqi kurdistan',
 'derbyshire',
 'bahrain grand prix',
 'consumer society',
 'doner kebab',
 'khao yai',
 'sea ice',
 'naaja nathanielsen',
 'wiretapping',
 'justice reform',
 'deutsche welle global media forum',
 'jerusalem day',
 'h.c. andersens hus',
 '\u2002guinea',
 'coronavirus skeptics',
 'air incursions',
 'swedbank',
 'rupture',
 'military intervention',
 'solar eclipse',
 'innocence in danger',
 'minneapolis',
 'biosphere',
 'eating disorders',
 'separatist party',
 'thai king',
 'persian',
 'zeruya shalev',
 'william turner',
 'dover',
 'fatherhood',
 'j-league',
 'thomas sankara',
 'christian drosten',
 'chapter four',
 'maraike aden',
 'legal system',
 'patron saint',
 'piedmont',
 'al hol',
 'international media law',
 'cyber intrusion',
 'corporate greed',
 'facebooh',
 'africavenir',
 'caretaker government',
 'lebanon crisis',
 'aldo davila',
 'new rules',
 'vulture funds',
 'joko widodo',
 'lone wolf',
 'grenfell',
 'a

In [115]:
list_kw_todel

[10031, 10393, 12554, 16927, 22742, 25184, 27616]

In [83]:
keywords_clean[10392:10394]

['baltic', 'suspilne, broadcaster, public broadcaster']

In [91]:
keywords_clean[10393].split(', ')

['suspilne', 'broadcaster', 'public broadcaster']

In [48]:
keywords = list(uni_kw['keyword'])

In [49]:
keywords.lower()


AttributeError: 'list' object has no attribute 'lower'

In [6]:
ratio_array= pr.cdist(list(uni_kw['keyword']), list(uni_kw['keyword']), score_cutoff = 90)

In [21]:
df_array = pd.DataFrame(ratio_array, columns = list(uni_kw['keyword']))
df_array.head(6)

,malware,state repression,climate consensus,Canary Islands,Angela I,Evan Rachel Wood,used cars,woodcarving,taboo,Grüne Woche,...,Drachen Lord,Missionaries,kiss-in,Fintech,Manuel Noriega,nuclear arms,processionary caterpillars,Deutsche Kammerphilharmonie Bremen,Maduro,self-sufficiency
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Count number of non zero values in each row
nb_non_zero = np.count_nonzero(np.asarray(ratio_array), axis=1) 

In [18]:
# Save indices of rows with more than 1 non-zero value
indices_correlating_rows = [i for i, el in enumerate(list(nb_non_zero)) if el>1]

In [30]:
keywords = list(uni_kw['keyword'])

In [44]:
len(indices_correlating_rows)

6084

In [45]:
for i in range(0,10):
    list_similar_words = [keyword for val, keyword in zip(list(df_array.iloc[indices_correlating_rows[i]]), keywords) if val!=0]
    print(list_similar_words)

['taboo', 'taboos']
['Qassem Soleimani', 'Qassem Soleimaini', 'Qassam Soleimani', 'Ghassem Soleimani']
['Boko haram', 'Boko Haram']
['Charlemagne Prize', 'Charlamagne Prize']
['Yahya Jammeh', 'Yayah Jammeh']
['Mosambique', 'Mozambique']
['Mikhael Gorbachev', 'Mikhail Gorbatchev', 'Mikhail Gorbachev']
['Domestic violence', 'domestic violence', 'Domestic Violence']
['global warming.', 'global warming']
['Dominican Republic', ', Dominican Republic']
